In [1]:
import pandas as pd
import numpy as np
import wbgapi as wb
from pandas_profiling import ProfileReport
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Données sur l'instabilité politique

df_insta_pol = pd.read_excel('../p5v2018.xls')


df_insta_pol = df_insta_pol[df_insta_pol['year']>= 1989][['country','year','polity2']] #de 1989 à 2018

# On harmonise le nom de certains pays
cntry_uni = [('Myanmar (Burma)','Myanmar')]

for cntry1,cntry2 in cntry_uni: 
    df_insta_pol[df_insta_pol.country == cntry1] = df_insta_pol[df_insta_pol.country == cntry1].assign(country=cntry2)

# On remplit les valeurs manquantes pour chaque pays avec les moyennes des valeurs dont on dispose pour ce pays
for cntry in df_insta_pol.country.unique():
    
    filler = df_insta_pol[df_insta_pol.country == cntry]['polity2'].mean()
    
    df_insta_pol[df_insta_pol.country == cntry] = df_insta_pol[df_insta_pol.country == cntry].fillna(value={'polity2': filler})

In [3]:
# Création d'une deuxième base comportant des variables par pays et année que l'on utilisera dans la partie modélisation

#Population totale, PIB par habitant (en $), Indice GINI, Croissance annuelle du PIB, taux de chomage, mortalité infantile, indice de la valeur des exportations, IDE
df_wb= wb.data.DataFrame(['SP.POP.TOTL','NY.GDP.MKTP.KD.ZG','SH.DTH.IMRT','TX.VAL.MRCH.XD.WD'], labels =True, time = range(1989, 2019), columns = 'series').reset_index().rename(columns = {'Time': 'year', 'SP.POP.TOTL' : 'population', 'NY.GDP.MKTP.KD.ZG' : 'croissancePIB', 'SH.DTH.IMRT' : 'mortalite-enf', 'TX.VAL.MRCH.XD.WD' : 'export'}).drop(columns = ['time','economy'])

In [4]:
df_wb.rename(columns = {'Country':'country'}, inplace = True)
df_wb.year = df_wb.year.astype(int)

# On harmonise le nom de certains pays
cntry_uni = [('Syrian Arab Republic','Syria'),('Yemen, Rep.','Yemen'),('Iran, Islamic Rep.','Iran'),('Egypt, Arab Rep.','Egypt')]

for cntry1,cntry2 in cntry_uni: 
    df_wb[df_wb.country == cntry1] = df_wb[df_wb.country == cntry1].assign(country=cntry2)
    
# On remplit les valeurs manquantes avec des moyennes 
for cntry in df_wb.country.unique():
    
    filler1 = df_wb[df_wb.country == cntry]['croissancePIB'].mean()
    filler2 = df_wb[df_wb.country == cntry]['mortalite-enf'].mean()
    filler3 = df_wb[df_wb.country == cntry]['population'].mean()
    filler4 = df_wb[df_wb.country == cntry]['export'].mean()
    
    df_wb[df_wb.country == cntry] =  df_wb[df_wb.country == cntry].fillna(value={'croissancePIB': filler,'mortalite-enf': filler2,'population': filler3,'export': filler4})

#On garde un data frame sans valeurs manquantes
df_wb = df_wb.dropna()

In [5]:
#On affiche la base pour le Yemen
df_insta_pol[df_insta_pol.country == 'Yemen']

,country,year,polity2
17281,Yemen,1990,0.0
17282,Yemen,1991,0.0
17283,Yemen,1992,0.0
17284,Yemen,1993,-2.0
17285,Yemen,1994,-2.0
17286,Yemen,1995,-2.0
17287,Yemen,1996,-2.0
17288,Yemen,1997,-2.0
17289,Yemen,1998,-2.0
17290,Yemen,1999,-2.0


In [6]:
# On s'assure que les pays sur lesquelles on va faire la jointure soient bien les mêmes dans les df
cntry_list = df_insta_pol.country.unique()
data_wb = df_wb[df_wb.country.isin(cntry_list)]
cntry_list2 = data_wb.country.unique()
data_pol = df_insta_pol[df_insta_pol.country.isin(cntry_list2)]

In [7]:
df_merge1 = pd.merge(data_wb, data_pol, on=['country', 'year'])

# On assigne 0 par défaut pour le ratio avant de le remplir
df_merge1['ratio-zone'] = 0
df_merge1['ratio'] = 0
df_merge1.year = df_merge1.year.astype(str)

In [8]:
#Visualisation du data frame
df_merge1

,country,year,croissancePIB,mortalite-enf,population,export,polity2,ratio-zone,ratio
0,Zimbabwe,2018,4.824211,17196.0,14438812.0,210.771948,4.0,0,0
1,Zimbabwe,2017,4.709492,18018.0,14236599.0,180.807792,4.0,0,0
2,Zimbabwe,2016,0.755869,18753.0,14030338.0,173.230130,4.0,0,0
3,Zimbabwe,2015,1.779873,19665.0,13814642.0,168.739718,4.0,0,0
4,Zimbabwe,2014,2.376929,20245.0,13586710.0,200.850799,4.0,0,0
...,...,...,...,...,...,...,...,...,...
4251,Afghanistan,1993,-2.333333,80103.0,15816601.0,57.297297,0.0,0,0
4252,Afghanistan,1992,-2.333333,78228.0,14485543.0,70.810811,0.0,0,0
4253,Afghanistan,1991,-2.333333,76732.0,13299016.0,101.621622,-8.0,0,0
4254,Afghanistan,1990,-2.333333,75864.0,12412311.0,127.081081,-8.0,0,0


In [10]:
# L'indicateur utilisé est la proportion qu'occupe un pays donné dans le nombre total de mort liées à un conflit au cours d'une année donnée
# C'est l'indicateur que l'on va chercher à expliquer

df_ucdp = pd.read_csv('../ucdp6.csv')

def ratio_best(country,y):
    # Nombre d'années que le conflit est en activité durant jusqu'à l'année 
        
    return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()

for y in df_ucdp['year'].unique():
    for cntry in df_ucdp[df_ucdp['year'] == y]['country'].unique():
        
        i = df_merge1[df_merge1.country == cntry][df_merge1.year == str(y)].index.values
        df_merge1.at[i,'ratio'] = ratio_best(cntry,y)

In [11]:
df_merge1

,country,year,croissancePIB,mortalite-enf,population,export,polity2,ratio-zone,ratio
0,Zimbabwe,2018,4.824211,17196.0,14438812.0,210.771948,4.0,0,0.000000
1,Zimbabwe,2017,4.709492,18018.0,14236599.0,180.807792,4.0,0,0.000000
2,Zimbabwe,2016,0.755869,18753.0,14030338.0,173.230130,4.0,0,0.000000
3,Zimbabwe,2015,1.779873,19665.0,13814642.0,168.739718,4.0,0,0.000000
4,Zimbabwe,2014,2.376929,20245.0,13586710.0,200.850799,4.0,0,0.000000
...,...,...,...,...,...,...,...,...,...
4251,Afghanistan,1993,-2.333333,80103.0,15816601.0,57.297297,0.0,0,0.061020
4252,Afghanistan,1992,-2.333333,78228.0,14485543.0,70.810811,0.0,0,0.057641
4253,Afghanistan,1991,-2.333333,76732.0,13299016.0,101.621622,-8.0,0,0.042159
4254,Afghanistan,1990,-2.333333,75864.0,12412311.0,127.081081,-8.0,0,0.015828


In [13]:
#On récupère la liste des pays frontaliers

df_border = pd.read_csv('../border.csv')
df_border.country_name.unique()

# On harmonise le nom de certains pays
cntry_uni = [('Bolivia (Plurinational State Of)','Bolivia'), ("Iran (Islamic Republic of)",'Iran'), ('Syrian Arab Republic','Syria'),("Venezuela (Bolivarian Republic of)",'Venezuela')]

for cntry1,cntry2 in cntry_uni: 
    df_border[df_border.country_name == cntry1] = df_border[df_border.country_name == cntry1].assign(country_name=cntry2)
    df_border[df_border.country_border_name == cntry1] = df_border[df_border.country_border_name == cntry1].assign(country_border_name=cntry2)

In [14]:
#On construit un indice qui calcule la somme des ratios des pays voisins frontaliers
def getregion_ratio(country,year):
    neighbors = df_border[df_border.country_name == country].country_border_name.values
    
    return df_merge1[df_merge1.year == year][df_merge1.country.isin(neighbors)].ratio.sum()

In [15]:
getregion_ratio('Afghanistan','2018')

0.006724348179548225

In [16]:
#On joint ce nouveau data frame à notre précédent data frame
for y in df_merge1['year'].unique():
    for cntry in df_merge1[df_merge1['year'] == y]['country'].unique():
        
        i = df_merge1[df_merge1.country == cntry][df_merge1.year == y].index.values
        df_merge1.at[i,'ratio-zone'] = getregion_ratio(cntry,y)

In [17]:
#Visualisation du data frame
df_merge1

,country,year,croissancePIB,mortalite-enf,population,export,polity2,ratio-zone,ratio
0,Zimbabwe,2018,4.824211,17196.0,14438812.0,210.771948,4.0,0.002021,0.000000
1,Zimbabwe,2017,4.709492,18018.0,14236599.0,180.807792,4.0,0.000029,0.000000
2,Zimbabwe,2016,0.755869,18753.0,14030338.0,173.230130,4.0,0.000631,0.000000
3,Zimbabwe,2015,1.779873,19665.0,13814642.0,168.739718,4.0,0.000023,0.000000
4,Zimbabwe,2014,2.376929,20245.0,13586710.0,200.850799,4.0,0.000201,0.000000
...,...,...,...,...,...,...,...,...,...
4251,Afghanistan,1993,-2.333333,80103.0,15816601.0,57.297297,0.0,0.018522,0.061020
4252,Afghanistan,1992,-2.333333,78228.0,14485543.0,70.810811,0.0,0.051726,0.057641
4253,Afghanistan,1991,-2.333333,76732.0,13299016.0,101.621622,-8.0,0.001258,0.042159
4254,Afghanistan,1990,-2.333333,75864.0,12412311.0,127.081081,-8.0,0.004589,0.015828


In [236]:
# On exporte finalement les données rassemblées 
df_merge1.to_csv('../df_regression.csv', index=False) 